# THESIS EXPERIMENTS

# Section 1: Setup

In [ ]:
# Import useful stuff
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM

from scipy.stats import binom, chi2
from scipy.io import arff

from evaluate_ExCeeD_alt import *
from ExCeeD_alt import *

import math
import os
import time
import random

In [ ]:
# Define functions
def create_data_point(dists):
    """
    Create a data point with features given by input distribution parameters.
    
    Parameters:
    dists: A list of distributions given by lists or tuples that define the mean and standard deviation of each of
           the random variables - e.g. [0,1] for a value drawn from a normal distribution with mean 0 and standard
           deviation 1.
           
    Return: A list representing a data point on the form [x1, x2,...m xn] where n is the number of distributions defined
            in the input dists
    
    """
    
    data_point = []
    for dist in dists:
        x = np.random.normal(dist[0], dist[1])
        data_point.append(x)
    
    return data_point
        
def create_data_set(dists, sample_size):
    """
    Create a data set of size sample_size by using the create_data_point function.
    
    Parameters:
    dists:         A list of distributions given by lists or tuples that define the mean and standard deviation of each of
                   the random variables - e.g. [0,1] for a value drawn from a normal distribution with mean 0 and standard
                   deviation 1.
    sample_size:   The number of data points in the data set
    
    Returns: a list of data points. Data points defined as described in the create_data_point function
    """
    data_points = []
    for n in range(sample_size):
        data_point = create_data_point(dists)
        data_points.append(data_point)
    
    return data_points

def compute_outlier_scores(df, outlier_method='lof'):
    """
    Compute outlier scores for each point in a data frame using one of four outlier detection techniques.
    
    Parameters:
    df:              DataFrame containing data to fit outlier detectio model to and then give outlier scores for.
    outlier_method:  Which outlier detection method to use. The following are implemented:
                         LocalOutlierFactor: to use LocalOutlierFactor use 'lof' as argument
                         k-NearestNeighbors: to use k-NearestNeighbors use 'knn' as argument
                         IsolationForest: to use IsolationForest use 'iforest' as argument
                         One-Class Support Vector Machine: to use One-Class Support Vector Machine use 'ocsvm' as argument
                         
    Returns: List of outlier scores for all rows in df                     
    """
    if outlier_method == 'lof':
        lof = LocalOutlierFactor()
        lof.fit_predict(df) 
        lof_scores = lof.negative_outlier_factor_   
        lof_scores *= -1
        outlier_scores = lof_scores
    elif outlier_method == 'knn':
        knn = KNN()
        knn.fit(df)
        knn_scores = knn.decision_scores_
        outlier_scores = knn_scores
    elif outlier_method == 'iforest':
        iforest = IForest()
        iforest.fit(df)
        iforest_scores = iforest.decision_scores_
        outlier_scores = iforest_scores
    elif outlier_method == 'ocsvm':
        ocsvm = OCSVM()
        ocsvm.fit(df)
        ocsvm_scores = ocsvm.decision_scores_
        outlier_scores = ocsvm_scores
    
    return outlier_scores

def cantelli_inequality(a, X):
    """
    Use the Cantelli Inequality to compute lower outlier probability bound for a point a from the mean of distribution X
    
    Parameters:
    a:    the distance from point, x, for which the lower outlier probability bound is to be computed and the mean of X
    X:    the distribution from which x is drawn
    
    Returns: the lower outlier probability bound of x (as a float)
    """
    var = np.var(X)
    k = a/math.sqrt(var)
    if k > 0:
        p = min(1, (1/(1+k**2)))
    else:
        p = 1
    return 1-p

def compute_cantelli_probability_bounds(df, column):
    """
    Use the cantelli_inequality function to compute lower outlier probability bounds for each row in column in
    df based on the Cantelli Inequality
    
    Parameters:
    df:     DataFrame from which the data is taken
    column: String name of the column in df to use as distribution for calculation of probability bounds
    
    Returns: a list containing outlier probability bounds for each row in df
    """
    props = []
    dist_mean = df[column].mean()
    for i,row in df.iterrows():
        pr = cantelli_inequality(row[column]-dist_mean, df[column])
        props.append(pr)
    
    return props

def calculateMahalanobis(data_point, mu, cov):
    """
    Calculate the mahalanobis distance from point data_point to the mean of the distribution.
    
    Parameters:
    data_point:    The data point for which to calculate mahalanobis distance
    distribution:  The distribution from wich data_point is drawn
    cov:           The covariance matrix of the distriution
    
    Returns: The mahalanobis distance from data_point to the mean of distriution
    """
    inv_cov = np.linalg.inv(cov)
    diff_from_mean = data_point - mu
    left = np.dot(diff_from_mean, inv_cov)
    inner = np.dot(left, diff_from_mean)
    d = math.sqrt(inner)
    
    return d

def compute_mahalanobis_distance(df, cov=None):
    """
    Use the calculateMahalanobis function to get the mahalanobis distance to the mean for all data points in df
    
    Parameters:
    df:    DataFrame where each row represents a data point for which to compute the mahalanobis distance to the mean
    cov:   Covariance matrix for the distribution from which x is drawn. If undefined, the function will define cov as
    the covariance matrix for df.
    """
    dists = []
    if not cov:
        cov = np.cov(df, rowvar=False)
        
    for i,row in df.iterrows():
        d = calculateMahalanobis(row, df.mean(), cov)
        dists.append(d)
    
    return dists


# Section 2: How does the distribution of outlier probabilities look for the cantelli-approach?
## Experiment 1

### - Both for 2-dimensional data sets constructed for this purpose and the data sets in the ExCeeD experiments and for all four outlier detection methods

This section contains experiments designed to see how the distribution of outlier probabilities look when using the Cantelli Inequality to evaluate outlier probabilities. This is done for several data sets and for four different outlier detection methods for each data set.

In [ ]:
# Create 2-dimensinal dataset
dists = [(0,1),(0,1)]
data = create_data_set(dists, 500)
df = pd.DataFrame(data, columns=['x1','x2'])

In [ ]:
# Compute outlier scores and outlier probabilities (according to the Cantelli Inequality)
df_copy = df.copy()
df['lof_score'] = compute_outlier_scores(df_copy, 'lof')
df['lof_prob (cant)'] = compute_cantelli_probability_bounds(df, 'lof_score')
df['knn_score'] = compute_outlier_scores(df_copy, 'knn')
df['knn_prob (cant)'] = compute_cantelli_probability_bounds(df, 'knn_score')
df['iforest_score'] = compute_outlier_scores(df_copy, 'iforest')
df['iforest_prob (cant)'] = compute_cantelli_probability_bounds(df, 'iforest_score')
df['ocsvm_score'] = compute_outlier_scores(df_copy, 'ocsvm')
df['ocsvm_prob (cant)'] = compute_cantelli_probability_bounds(df, 'ocsvm_score')
df

In [ ]:
# Plot histograms to visualize distribution of outlier scores
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(figsize=(10, 6), nrows=2, ncols=2)

ax1.hist(df['lof_score'])
ax1.set_title('LOF')
ax1.set_xlabel('score')
ax1.set_ylabel('count')

ax2.hist(df['knn_score'])
ax2.set_title('kNN')
ax2.set_xlabel('score')
ax2.set_ylabel('count')

ax3.hist(df['iforest_score'])
ax3.set_title('IForest')
ax3.set_xlabel('score')
ax3.set_ylabel('count')

ax4.hist(df['ocsvm_score'])
ax4.set_title('OCSVM')
ax4.set_xlabel('score')
ax4.set_ylabel('count')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment1-1')

In [ ]:
# Visualize outlier probability for each outlier detection method
fig, axes = plt.subplots(figsize=(10,6), nrows=2, ncols=2)

axes[0,0].scatter(df['lof_score'], df['lof_prob (cant)'])
axes[0,0].set_title('LOF')
axes[0,0].set_xlabel('score')
axes[0,0].set_ylabel('outlier probability')

axes[0,1].scatter(df['knn_score'], df['knn_prob (cant)'])
axes[0,1].set_title('kNN')
axes[0,1].set_xlabel('score')
axes[0,1].set_ylabel('outlier probability')

axes[1,0].scatter(df['iforest_score'], df['iforest_prob (cant)'])
axes[1,0].set_title('IForest')
axes[1,0].set_xlabel('score')
axes[1,0].set_ylabel('outlier probability')

axes[1,1].scatter(df['ocsvm_score'], df['ocsvm_prob (cant)'])
axes[1,1].set_title('OCSVM')
axes[1,1].set_xlabel('score')
axes[1,1].set_ylabel('outlier probability')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment1-2')

In [ ]:
# find fraction of outlier scores that are above 0.80 or below 0.20 for the four outlier detection methods
lof_frac = df[(df['lof_prob (cant)'] >= 0.8) | (df['lof_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
knn_frac = df[(df['knn_prob (cant)'] >= 0.8) | (df['knn_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
iforest_frac = df[(df['iforest_prob (cant)'] >= 0.8) | (df['iforest_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
ocsvm_frac = df[(df['ocsvm_prob (cant)'] >= 0.8) | (df['ocsvm_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]

# collect fractions in DataFrame
fracs = [[lof_frac, knn_frac, iforest_frac, ocsvm_frac]]
columns = ['LOF', 'kNN', 'IForest', 'OCSVM']
frac_df = pd.DataFrame(fracs, columns=columns)

# display DataFrame
frac_df

Repeat the above process with data set from "benchmark datasets"

In [ ]:
# Load data set into DataFrame
data = arff.loadarff('Benchmark_Datasets/WBC_norm_v02.arff') # WBC data set
df = pd.DataFrame(data[0])

In [ ]:
X = df[df.columns[:-2]].values #insert the effective features (no response label y)

In [ ]:
# Compute outlier scores and Cantelli outlier probabilities for the different outlier detection methods
df['lof_score'] = compute_outlier_scores(X, 'lof')
df['lof_prob (cant)'] = compute_cantelli_probability_bounds(df, 'lof_score')
df['knn_score'] = compute_outlier_scores(X, 'knn')
df['knn_prob (cant)'] = compute_cantelli_probability_bounds(df, 'knn_score')
df['iforest_score'] = compute_outlier_scores(X, 'iforest')
df['iforest_prob (cant)'] = compute_cantelli_probability_bounds(df, 'iforest_score')
df['ocsvm_score'] = compute_outlier_scores(X, 'ocsvm')
df['ocsvm_prob (cant)'] = compute_cantelli_probability_bounds(df, 'ocsvm_score')

In [ ]:
# Plot histograms to visualize distribution of outlier scores
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(figsize=(10, 6), nrows=2, ncols=2)

ax1.hist(df['lof_score'])
ax1.set_title('LOF')
ax1.set_xlabel('score')
ax1.set_ylabel('count')

ax2.hist(df['knn_score'])
ax2.set_title('kNN')
ax2.set_xlabel('score')
ax2.set_ylabel('count')

ax3.hist(df['iforest_score'])
ax3.set_title('IForest')
ax3.set_xlabel('score')
ax3.set_ylabel('count')

ax4.hist(df['ocsvm_score'])
ax4.set_title('OCSVM')
ax4.set_xlabel('score')
ax4.set_ylabel('count')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment1-3')

In [ ]:
# Visualize outlier probability for each outlier detection method
fig, axes = plt.subplots(figsize=(10,6), nrows=2, ncols=2)

axes[0,0].scatter(df['lof_score'], df['lof_prob (cant)'])
axes[0,0].set_title('LOF')
axes[0,0].set_xlabel('score')
axes[0,0].set_ylabel('outlier probability')

axes[0,1].scatter(df['knn_score'], df['knn_prob (cant)'])
axes[0,1].set_title('kNN')
axes[0,1].set_xlabel('score')
axes[0,1].set_ylabel('outlier probability')

axes[1,0].scatter(df['iforest_score'], df['iforest_prob (cant)'])
axes[1,0].set_title('IForest')
axes[1,0].set_xlabel('score')
axes[1,0].set_ylabel('outlier probability')

axes[1,1].scatter(df['ocsvm_score'], df['ocsvm_prob (cant)'])
axes[1,1].set_title('OCSVM')
axes[1,1].set_xlabel('score')
axes[1,1].set_ylabel('outlier probability')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment1-4')

In [ ]:
# find fraction of outlier scores that are above 0.80 or below 0.20 for the four outlier detection methods
frac_df_small = frac_df.copy()
lof_frac = df[(df['lof_prob (cant)'] >= 0.8) | (df['lof_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
knn_frac = df[(df['knn_prob (cant)'] >= 0.8) | (df['knn_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
iforest_frac = df[(df['iforest_prob (cant)'] >= 0.8) | (df['iforest_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
ocsvm_frac = df[(df['ocsvm_prob (cant)'] >= 0.8) | (df['ocsvm_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]

# collect fractions in DataFrame
fracs = [lof_frac, knn_frac, iforest_frac, ocsvm_frac]
frac_df_small.loc[frac_df_small.shape[0]] = fracs
frac_df_small.rename(index={0:'Generated Data', 1:'Benchmark Dataset'}, inplace=True)

# display DataFrame
frac_df_small

In [ ]:
# Make plots for all benchmark data sets
indexes = ['Generated Data']
datasets = os.listdir('Benchmark_Datasets/')[1:]
for dataset in datasets:
    data = arff.loadarff('Benchmark_Datasets/'+dataset)
    df = pd.DataFrame(data[0])
    X = df[df.columns[:-2]].values #insert the effective features (no response label y)
    df['lof_score'] = compute_outlier_scores(X, 'lof')
    df['lof_prob (cant)'] = compute_cantelli_probability_bounds(df, 'lof_score')
    df['knn_score'] = compute_outlier_scores(X, 'knn')
    df['knn_prob (cant)'] = compute_cantelli_probability_bounds(df, 'knn_score')
    df['iforest_score'] = compute_outlier_scores(X, 'iforest')
    df['iforest_prob (cant)'] = compute_cantelli_probability_bounds(df, 'iforest_score')
    df['ocsvm_score'] = compute_outlier_scores(X, 'ocsvm')
    df['ocsvm_prob (cant)'] = compute_cantelli_probability_bounds(df, 'ocsvm_score')
    
    fig, axes = plt.subplots(2,2)

    # Visualize outlier probability for each outlier detection method
    fig, axes = plt.subplots(2,2)

    axes[0,0].scatter(df['lof_score'], df['lof_prob (cant)'])
    axes[0,0].set_title('LOF')
    axes[0,1].scatter(df['knn_score'], df['knn_prob (cant)'])
    axes[0,1].set_title('kNN')
    axes[1,0].scatter(df['iforest_score'], df['iforest_prob (cant)'])
    axes[1,0].set_title('IForest')
    axes[1,1].scatter(df['ocsvm_score'], df['ocsvm_prob (cant)'])
    axes[1,1].set_title('OCSVM')
    fig.suptitle(dataset[:-5], fontsize=16)
    fig.tight_layout()
    #fig.savefig('images/appendix/experiment1-'+dataset[:-5])
    
    lof_frac = df[(df['lof_prob (cant)'] >= 0.8) | (df['lof_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
    knn_frac = df[(df['knn_prob (cant)'] >= 0.8) | (df['knn_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
    iforest_frac = df[(df['iforest_prob (cant)'] >= 0.8) | (df['iforest_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]
    ocsvm_frac = df[(df['ocsvm_prob (cant)'] >= 0.8) | (df['ocsvm_prob (cant)'] <= 0.2)].shape[0]/df.shape[0]

    fracs = [lof_frac, knn_frac, iforest_frac, ocsvm_frac]
    indexes.append(dataset[:-5])
    
    frac_df.loc[frac_df.shape[0]] = fracs

frac_df.rename(index={i:name for i,name in enumerate(indexes)}, inplace=True)
frac_df

Results above look kinda as expected and based on these results the Cantelli Inequality seem like a reasonable method for evaluating outlier probabilities (it serves as noralization and behaves in a fairly reasonable way).

# Section 3: Cantelli-ExCeeD
## Experiment 2
### - including the Cantelli-approach as outlier socre interpretation method and repeat the experiments from the Perini paper

This section contains experiments designed to evaluate the Cantelli method against the ExCeeD method. This involves running an altered version of the evaluate_ExCeeD script from the Perini paper where the Cantelli method is included as an additional outlier probability interpretation method. This would then be the Cantelli-ExCeeD

In [ ]:
datasets = os.listdir('Benchmark_Datasets/')[2:]
results = []
i = len(datasets)
t_start = time.time()
for dataset in datasets:
    i -=1
    print(dataset[:-5])
    data_path = 'Benchmark_Datasets/'+dataset
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    
    print(f'shape: {df.shape}')
    
    df['outlier'] = [string.decode("utf-8") for string in df['outlier'].values]
    y = np.asarray([1 if string == 'yes' else 0 for string in df['outlier'].values])
    X = df[df.columns[:-2]].values #insert the effective features (no response label y)

    model = 'KNN' #otherwise 'IForest', 'OCSVM'

    L2_error = compute_confidence_error(model, X, y, saveresults = True) #dictionary with the L2 errors

    results.append(L2_error)
    
    t_end = time.time()
    print(f'finished in {t_end - t_start} seconds')
    print(f'remaining datasets: {i}')
    
df_results_knn = pd.concat(results)
dataset_names = [dataset[:-5] for dataset in datasets]
df_results_knn.index = dataset_names
df_results_knn

In [ ]:
results = []
i = len(datasets)
t_start = time.time()
for dataset in datasets:
    i -=1
    print(dataset[:-5])
    data_path = 'Benchmark_Datasets/'+dataset
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    
    print(f'shape: {df.shape}')
    
    df['outlier'] = [string.decode("utf-8") for string in df['outlier'].values]
    y = np.asarray([1 if string == 'yes' else 0 for string in df['outlier'].values])
    X = df[df.columns[:-2]].values #insert the effective features (no response label y)

    model = 'IForest' #otherwise 'KNN', 'OCSVM'

    L2_error = compute_confidence_error(model, X, y, saveresults = True) #dictionary with the L2 errors

    results.append(L2_error)
    
    t_end = time.time()
    print(f'finished in {t_end - t_start} seconds')
    print(f'remaining datasets: {i}')
    
df_results_iforest = pd.concat(results)
df_results_iforest.index = dataset_names
df_results_iforest

In [ ]:
results = []
i = len(datasets)
t_start = time.time()
for dataset in datasets:
    i -=1
    print(dataset[:-5])
    data_path = 'Benchmark_Datasets/'+dataset
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    
    print(f'shape: {df.shape}')
    
    df['outlier'] = [string.decode("utf-8") for string in df['outlier'].values]
    y = np.asarray([1 if string == 'yes' else 0 for string in df['outlier'].values])
    X = df[df.columns[:-2]].values #insert the effective features (no response label y)

    model = 'OCSVM' #otherwise 'KNN', 'IForest'

    L2_error = compute_confidence_error(model, X, y, saveresults = True) #dictionary with the L2 errors

    results.append(L2_error)
    
    t_end = time.time()
    print(f'finished in {t_end - t_start} seconds')
    print(f'remaining datasets: {i}')
    
df_results_ocsvm = pd.concat(results)
df_results_ocsvm.index = dataset_names
df_results_ocsvm

In [ ]:
# Summarize results - how often does the ExCeeD method beat each of the competitors?
df_summary = pd.DataFrame([], columns=['ExCeeD wins', 'ExCeeD loses', 'ExCeeD draws'])

competitors = df_results_ocsvm.columns[1:]

for c in competitors:
    
    win_count = 0
    draw_count = 0
    
    for df_results in [df_results_knn, df_results_iforest, df_results_ocsvm]:
        wins = df_results[c] < df_results['ExCeeD']
        draws = df_results[c] == df_results['ExCeeD']

        if True in wins.value_counts():
            win_count += wins.value_counts()[True]
        else:
            win_count += 0

        if True in draws.value_counts():
            draw_count += draws.value_counts()[True]
        else:
            draw_count += 0
        


    loss = 63 - (win_count + draw_count)
    row = [loss, win_count, draw_count]
    df_summary.loc[df_summary.shape[0]] = row

df_summary.rename(index={i:name for i,name in enumerate(competitors)}, inplace=True)
df_summary


In [ ]:
# mean and standard deciation of rank (based on error in confidence) for each of the methods over all data sets
ranks = [[],[],[],[],[],[],[],[],[],[]]
for df_results in [df_results_knn, df_results_iforest, df_results_ocsvm]:
    for i,row in df_results.iterrows():
        for j in range(10):
            ranks[j].append(row.rank()[j])
    
avg_ranks = []
for r in ranks:
    m = np.mean(r)
    sd = np.std(r)
    avg_ranks.append((m,sd))
    
avg_ranks

df_ranks = pd.DataFrame(avg_ranks, columns=['mean', 'standard deviation'])
df_ranks.rename(index={i+1:name for i,name in enumerate(competitors)}, inplace=True)
df_ranks.rename(index={0:'ExCeeD'}, inplace=True)
df_ranks

In [ ]:
# mean and standard deciation of error in confidence for each of the methods over all data sets
errors = [[],[],[],[],[],[],[],[],[],[]]
for df_results in [df_results_knn, df_results_iforest, df_results_ocsvm]:
    for i,row in df_results.iterrows():
        for j in range(10):
            errors[j].append(row[j])
    
avg_errors = []
for e in errors:
    m = np.mean(e)
    sd = np.std(e)
    avg_errors.append((m*100,sd*100))
    
avg_errors

df_errors = pd.DataFrame(avg_errors, columns=['mean', 'standard deviation'])
df_errors.rename(index={i+1:name for i,name in enumerate(competitors)}, inplace=True)
df_errors.rename(index={0:'ExCeeD'}, inplace=True)
df_errors

In [ ]:
# how often does the Cantelli method beat each of the competitors?
df_summary = pd.DataFrame([], columns=['ExCeeD_cant wins', 'ExCeeD_cant loses', 'ExCeeD_cant draws'])

competitors = df_results_ocsvm.columns[2:]

for c in competitors:
    
    win_count = 0
    draw_count = 0
    
    for df_results in [df_results_knn, df_results_iforest, df_results_ocsvm]:
        wins = df_results[c] < df_results['ExCeeD_cant']
        draws = df_results[c] == df_results['ExCeeD_cant']

        if True in wins.value_counts():
            win_count += wins.value_counts()[True]
        else:
            win_count += 0

        if True in draws.value_counts():
            draw_count += draws.value_counts()[True]
        else:
            draw_count += 0
        


    loss = 63 - (win_count + draw_count)
    row = [loss, win_count, draw_count]
    df_summary.loc[df_summary.shape[0]] = row

df_summary.rename(index={i:name for i,name in enumerate(competitors)}, inplace=True)
df_summary

# Section 4: Compare output of ExCeeD and Cantelli
## Experiment 3

This section contains some plots to compare the resulting outlier probabilities and confidences of the ExCeeD method and the Cantelli method

In [ ]:
# Define data set
datasets = os.listdir('Benchmark_Datasets/')[1:] # drop first element in list (notebook checkpoint folder)
dataset = datasets[-4] # Select data set - -4 is WBC
data_path = 'Benchmark_Datasets/'+dataset

In [ ]:
# Load and prepare data set
data = arff.loadarff(data_path) # Load selected data set
df = pd.DataFrame(data[0])
df['outlier'] = [string.decode("utf-8") for string in df['outlier'].values]
y = np.asarray([1 if string == 'yes' else 0 for string in df['outlier'].values])
X = df[df.columns[:-2]].values #insert the effective features (no response label y)

n = np.shape(X)[0]
contamination = sum(y)/len(y)
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=331)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# fit kNN outlier detector and get scores    
knno = KNN(n_neighbors=np.int(n*contamination), contamination = contamination).fit(X_train)
train_scores_knno = knno.decision_function(X_train)
test_scores_knno = knno.decision_function(X_test)
prediction_knno = knno.predict(X_test)

In [ ]:
# Compute ExCeeD and Cantelli probabilities and confidences
exceed_prob, exceed_conf = ExCeeD_alt(train_scores_knno, test_scores_knno, prediction_knno, contamination, alg='bayes')
exceed_cant_prob, exceed_cant_conf = ExCeeD_alt(train_scores_knno, test_scores_knno, prediction_knno, contamination, alg='cant')

In [ ]:
# create DataFrame with score, probabilities, and confidences
data = [test_scores_knno, exceed_prob, exceed_conf, exceed_cant_prob, exceed_cant_conf]
df = pd.DataFrame(data).transpose()
df.columns = ['Scores', 'ExCeeD_prob', 'ExCeeD_conf', 'Cant_prob', 'Cant_conf']

In [ ]:
# Visualize probabilities and confidences
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), nrows=1, ncols=2)
ax1.scatter(x=df['Scores'], y=df['ExCeeD_prob'], label='ExCeeD')
ax1.scatter(x=df['Scores'], y=df['Cant_prob'], label='Cantelli')
ax1.set_title('Outlier Probability')
ax1.set_xlabel('Score')
ax1.set_ylabel('Outlier Probability')
ax1.legend(loc='upper left')

diff = [df['ExCeeD_conf'][i] - df['Cant_conf'][i] for i in range(df.shape[0])]
ax2.scatter(x=df['Scores'], y=df['ExCeeD_conf'], label='ExCeeD', alpha=.6)
ax2.scatter(x=df['Scores'], y=df['Cant_conf'], label='Cantelli', alpha=.6)
ax2.scatter(x=df['Scores'], y=diff, label='Difference', alpha=.6)
ax2.set_title('Confidence')
ax2.set_xlabel('Score')
ax2.set_ylabel('Confidence')
ax2.legend(loc='center left')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment3-1')

In [ ]:
# Visualize probabilities and confidences
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), nrows=1, ncols=2)

ax1.scatter(x=df['Scores'], y=df['ExCeeD_prob'], label='Probability', alpha=.5)
ax1.scatter(x=df['Scores'], y=df['ExCeeD_conf'], label='Confidence', alpha=.5)
ax1.set_title('ExCeeD')
ax1.set_xlabel('Score')
ax1.set_ylabel('Probability/Confidence')
ax1.legend(loc='lower right')

ax2.scatter(x=df['Scores'], y=df['Cant_prob'], label='Probability', alpha=.5)
ax2.scatter(x=df['Scores'], y=df['Cant_conf'], label='Confidence', alpha=.5)
ax2.set_title('Cantelli')
ax2.set_xlabel('Score')
ax2.set_ylabel('Probability/Confidence')
ax2.legend(loc='center left')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment3-2')

##### Test on multivariate normal distribution

In [ ]:
# Create bimodal bivariate distribution
dists_test = [(0,1),(0,1)]
dists2 = [(2,2), (2,2.2)]
data_test = create_data_set(dists_test, 500)
data2 = create_data_set(dists2, 30)
data = data_test + data2
y = np.array([0 for i in range(500)] + [1 for i in range(30)])
data = np.array(data)
df = pd.DataFrame(data, columns=['x1','x2'])

In [ ]:
# Split into train and test set
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=331)
for train_index, test_index in sss.split(data, y):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index], y[test_index]
df_test = pd.DataFrame(X_test, columns=['x1','x2'])

In [ ]:
# Visualize data set
colors = np.array(['b' if i <= 500 else 'r' for i in range(df.shape[0])])
sc = plt.scatter(x=df['x1'], y=df['x2'], c=colors)
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-5, 5])
plt.grid(linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
# Visualize test set
colors = ['b' if e == 0 else 'r' for e in y_test]
sc = plt.scatter(x=df_test['x1'], y=df_test['x2'], c=colors)
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-5, 5])
plt.grid(linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
# Visualize data set and test set
fig, (ax1, ax2) = plt.subplots(figsize=(10,5), nrows=1, ncols=2)

colors1 = np.array(['b' if i <= 500 else 'r' for i in range(df.shape[0])])
colors2 = ['b' if e == 0 else 'r' for e in y_test]
           
ax1.scatter(x=df['x1'], y=df['x2'], c=colors1)
ax1.set_title('Data Set')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
xlim = ax1.get_xlim()
ylim = ax1.get_ylim()

ax2.scatter(x=df_test['x1'], y=df_test['x2'], c=colors2)
ax2.set_title('Test Set')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.set_xlim(xlim)
ax2.set_ylim(ylim)

#fig.savefig('images/experiment3-dataset-1')

In [ ]:
# Fit kNN and get scores
contamination = 30/530
knno = KNN(n_neighbors=np.int(530*contamination), contamination = contamination).fit(X_train)
train_scores_knno = knno.decision_function(X_train)
test_scores_knno = knno.decision_function(X_test)
prediction_knno = knno.predict(X_test)

In [ ]:
# Compute ExCeeD and Cantelli probabilities and confidences
exceed_prob, exceed_conf = ExCeeD_alt(train_scores_knno, test_scores_knno, prediction_knno, contamination, alg='bayes')
exceed_cant_prob, exceed_cant_conf = ExCeeD_alt(train_scores_knno, test_scores_knno, prediction_knno, contamination, alg='cant')

In [ ]:
exceed_prob = np.array(exceed_prob)
exceed_cant_prob = np.array(exceed_cant_prob)
prob_diff = exceed_prob-exceed_cant_prob

In [ ]:
# Visualize probabilities and confidences
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), nrows=1, ncols=2)
ax1.scatter(x=test_scores_knno, y=exceed_prob, label='ExCeeD')
ax1.scatter(x=test_scores_knno, y=exceed_cant_prob, label='Cantelli')
ax1.set_title('Outlier Probability')
ax1.set_xlabel('Score')
ax1.set_ylabel('Outlier Probability')
ax1.legend(loc='lower right')

diff = [exceed_conf[i] - exceed_cant_conf[i] for i,v in enumerate(exceed_conf)]
ax2.scatter(x=test_scores_knno, y=exceed_conf, label='ExCeeD', alpha=.6)
ax2.scatter(x=test_scores_knno, y=exceed_cant_conf, label='Cantelli', alpha=.6)
ax2.scatter(x=test_scores_knno, y=diff, label='Difference', alpha=.6)
ax2.set_title('Confidence')
ax2.set_xlabel('Score')
ax2.set_ylabel('Confidence')
ax2.legend(loc='lower right')

#fig.savefig('images/experiment3-3')

In [ ]:
# Visualize probabilities and confidences
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), nrows=1, ncols=2)
ax1.scatter(x=test_scores_knno, y=exceed_prob, label='Probability', alpha=.5)
ax1.scatter(x=test_scores_knno, y=exceed_conf, label='Confidence', alpha=.5)
ax1.set_title('ExCeeD')
ax1.set_xlabel('Score')
ax1.set_ylabel('Probability/Confidence')
ax1.legend(loc='lower right')

ax2.scatter(x=test_scores_knno, y=exceed_cant_prob, label='Probability', alpha=.5)
ax2.scatter(x=test_scores_knno, y=exceed_cant_conf, label='Confidence', alpha=.5)
ax2.set_title('Cantelli')
ax2.set_xlabel('Score')
ax2.set_ylabel('Probability/Confidence')
ax2.legend(loc='lower right')

#fig.savefig('images/experiment3-4')

In [ ]:
# Visualize probabilities and confidences
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(figsize=(12, 8), nrows=2, ncols=2)
ax1.scatter(x=test_scores_knno, y=exceed_prob, label='ExCeeD')
ax1.scatter(x=test_scores_knno, y=exceed_cant_prob, label='Cantelli')
ax1.set_title('Outlier Probability')
ax1.set_xlabel('Score')
ax1.set_ylabel('Outlier Probability')
ax1.legend(loc='lower right')

diff = [exceed_conf[i] - exceed_cant_conf[i] for i,v in enumerate(exceed_conf)]
ax2.scatter(x=test_scores_knno, y=exceed_conf, label='ExCeeD', alpha=.6)
ax2.scatter(x=test_scores_knno, y=exceed_cant_conf, label='Cantelli', alpha=.6)
ax2.scatter(x=test_scores_knno, y=diff, label='Difference', alpha=.6)
ax2.set_title('Confidence')
ax2.set_xlabel('Score')
ax2.set_ylabel('Confidence')
ax2.legend(loc='lower right')

ax3.scatter(x=test_scores_knno, y=exceed_prob, label='Probability', alpha=.5)
ax3.scatter(x=test_scores_knno, y=exceed_conf, label='Confidence', alpha=.5)
ax3.set_title('ExCeeD')
ax3.set_xlabel('Score')
ax3.set_ylabel('Probability/Confidence')
ax3.legend(loc='lower right')

ax4.scatter(x=test_scores_knno, y=exceed_cant_prob, label='Probability', alpha=.5)
ax4.scatter(x=test_scores_knno, y=exceed_cant_conf, label='Confidence', alpha=.5)
ax4.set_title('Cantelli')
ax4.set_xlabel('Score')
ax4.set_ylabel('Probability/Confidence')
ax4.legend(loc='lower right')

fig.tight_layout(pad=3)
#fig.savefig('images/experiment3-3')

In [ ]:
# Visualize differences in probabilities
colors1 = exceed_prob
colors2 = exceed_cant_prob
colors3 = prob_diff

colors = [colors1, colors2,colors3]

p_min = min(np.concatenate(colors))
p_max = max(np.concatenate(colors))

fig, (ax1, ax2, ax3) = plt.subplots(figsize=(10, 4), nrows=1, ncols=3)

im1 = ax1.scatter(x=df_test['x1'], y=df_test['x2'], c=colors1, edgecolors='black', cmap='hot_r', vmin=p_min, vmax=p_max)
im2 = ax2.scatter(x=df_test['x1'], y=df_test['x2'], c=colors2, edgecolors='black', cmap='hot_r', vmin=p_min, vmax=p_max)
im3 = ax3.scatter(x=df_test['x1'], y=df_test['x2'], c=colors3, edgecolors='black', cmap='hot_r', vmin=p_min, vmax=p_max)

ax1.set_title('ExCeeD')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax2.set_title('Cantelli')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax3.set_title('Difference')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')

ax1.grid(linestyle = '--', linewidth = 0.5)
ax2.grid(linestyle = '--', linewidth = 0.5)
ax3.grid(linestyle = '--', linewidth = 0.5)


cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im3, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment3-4', bbox_inches='tight')
#plt.show()

In [ ]:
# Compute true outlier probabilities
real_prob = []
dists = []
mu = np.array([0,0])
sigma = np.array([[1,0],[0,1]])
for x in X_test:
    m_dist_x = calculateMahalanobis(x, mu, sigma)
    p = chi2.cdf(m_dist_x, df=2)
    real_prob.append(p)

real_prob = np.array(real_prob)

In [ ]:
# Visualize true outlier probabilities as well as differences between ExCeeD/Cantelli and true
colors1 = real_prob
colors2 = exceed_prob - real_prob
colors3 = exceed_cant_prob - real_prob

colors = [colors1, colors2,colors3]

p_min = min(np.concatenate(colors))
p_max = max(np.concatenate(colors))

p_lim = max(abs(p_min), abs(p_max))

p_min = -1*p_lim
p_max = p_lim

titles = ['True', 'ExCeeD','Cantelli']

fig, axes = plt.subplots(figsize=(10, 4), nrows=1, ncols=3)

for i, ax in enumerate(axes.flat):
    im = ax.scatter(x=df_test['x1'], y=df_test['x2'], c=colors[i], edgecolors='black', cmap='seismic', vmin=p_min, vmax=p_max)
    ax.grid(linestyle = '--', linewidth = 0.5)
    ax.set_title(titles[i])
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')


cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment3-5', bbox_inches='tight')
#plt.show()

In [ ]:
# Visualize probabilities and confidences
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), nrows=1, ncols=2)
ax1.scatter(x=test_scores_knno, y=exceed_prob, label='ExCeeD')
ax1.scatter(x=test_scores_knno, y=exceed_cant_prob, label='Cantelli')
ax1.scatter(x=test_scores_knno, y=real_prob, label='True')
ax1.set_title('Outlier Probability')
ax1.set_xlabel('Outlier Score')
ax1.set_ylabel('Outlier Probability')
ax1.legend(loc='lower right')

ax2.scatter(x=test_scores_knno, y=exceed_conf, label='ExCeeD', alpha=.5)
ax2.scatter(x=test_scores_knno, y=exceed_cant_conf, label='Cantelli', alpha=.5)
ax2.set_title('Confidence')
ax2.set_xlabel('Score')
ax2.set_ylabel('Confidence')
ax2.legend(loc='lower right')

#fig.savefig('images/experiment3-6')

# Section 5: Investigate the ExCeeD method more in depth
## Experiment 4
### - Is outlier "rank" actually what matters?
### - How is the confidence expected to look?
This section contains experiments made to evaluate the ExCeeD method and better understand it

### Experiment: Does ExCeeD evaluate scores or rankings?

In [ ]:
# Create data set
dists1 = [(0,1),(0,1)]
size_dataset = 500
data_set1 = create_data_set(dists1, size_dataset)
df = pd.DataFrame(data_set1, columns=['x1','x2'])
df['dist'] = compute_mahalanobis_distance(df)
df['outlier score1'] = [d/max(df['dist']) for d in df['dist']]
df['outlier score2'] = [(d/max(df['dist']))**3 for d in df['dist']]

In [ ]:
contamination = 0.02
n_outliers = math.floor(contamination*size_dataset)
t = df['outlier score1'].nlargest(n=n_outliers).iloc[-1]
predictions = np.array([1 if s >= t else 0 for s in df['outlier score1']])

In [ ]:
# Visualize outlier scores
plt.scatter(df['dist'], df['outlier score1'])
plt.scatter(df['dist'], df['outlier score2'])

In [ ]:
# Visualizing the "outlier scores"
fig, (ax1,ax2, ax3) = plt.subplots(figsize=(12, 4),nrows=1,ncols=3)

color1 = df['outlier score1']
color2 = df['outlier score2']
colors = pd.concat([color1, color2])
p_min = min(colors)
p_max = max(colors)

ax1.scatter(df['dist'], df['outlier score1'], label='Linear')
ax1.scatter(df['dist'], df['outlier score2'], label='Cubed')
ax1.set_title('Outlier Scores')
ax1.set_xlabel('Mahalanobis Distance')
ax1.set_ylabel('Outlier Score')
ax1.legend(loc='upper left')
ax2.scatter(df['x1'], df['x2'], c=color1, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax2.set_title('Linear')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
im = ax3.scatter(df['x1'], df['x2'], c=color2, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax3.set_title('Cubed')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')

cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment-setup-4-1', bbox_inches='tight')

In [ ]:
# Compute ExCeeD and Cantelli probabilities and confidences
exceed_prob1_full, exceed_conf1_full = ExCeeD_alt(df['outlier score1'], df['outlier score1'], predictions, contamination, alg='bayes')
exceed_prob2_full, exceed_conf2_full = ExCeeD_alt(df['outlier score2'], df['outlier score2'], predictions, contamination, alg='bayes')

In [ ]:
colors1 = exceed_prob1_full
colors2 = exceed_prob2_full
colors = [colors1, colors2]
color_diff = np.array(colors1) - np.array(colors2)

In [ ]:
# Visualize outlier probabilities (and the difference)
fig, (ax1,ax2,ax3) = plt.subplots(figsize=(12, 4), nrows=1, ncols=3)
p_min = min(np.concatenate(colors))
p_max = max(np.concatenate(colors))
ax1.scatter(df['x1'], df['x2'], c=colors1, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax1.set_title('Linear')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax2.scatter(df['x1'], df['x2'], c=colors2, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax2.set_title('Cubed')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
im = ax3.scatter(df['x1'], df['x2'], c=color_diff, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax3.set_title('Difference')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')

cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment4-1', bbox_inches='tight')

There is no difference in estimated outlier probabilities for the points even though their outlier scores are guite different.

Does it change if I split the data set in train and test data? Two tests:
- 0.6/0.4 split
- 0.3/0.7 split

###### First: the 0.6/0.4 split

In [ ]:
# split data
train_scores_medium, test_scores_medium = train_test_split(df, test_size=0.6)

In [ ]:
contamination = 0.02
size_dataset = test_scores_medium.shape[0]
n_outliers = math.floor(contamination*size_dataset)
t = test_scores_medium['outlier score1'].nlargest(n=n_outliers).iloc[-1]
predictions = np.array([1 if s >= t else 0 for s in test_scores_medium['outlier score1']])

In [ ]:
# Compute ExCeeD and Cantelli probabilities and confidences
exceed_prob1_medium, exceed_conf1_medium = ExCeeD_alt(train_scores_medium['outlier score1'], test_scores_medium['outlier score1'], predictions, contamination, alg='bayes')
exceed_prob2_medium, exceed_conf2_medium = ExCeeD_alt(train_scores_medium['outlier score2'], test_scores_medium['outlier score2'], predictions, contamination, alg='bayes')

In [ ]:
colors1 = exceed_prob1_medium
colors2 = exceed_prob2_medium
colors = [colors1, colors2]
color_diff = np.array(colors1) - np.array(colors2)

In [ ]:
# Visualize outlier probabilities (and the difference)
fig, (ax1,ax2,ax3) = plt.subplots(figsize=(12, 4), nrows=1, ncols=3)
p_min = min(np.concatenate(colors))
p_max = max(np.concatenate(colors))
ax1.scatter(test_scores_medium['x1'], test_scores_medium['x2'], c=colors1, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax1.set_title('Linear')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax2.scatter(test_scores_medium['x1'], test_scores_medium['x2'], c=colors2, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax2.set_title('Cubed')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
im = ax3.scatter(test_scores_medium['x1'], test_scores_medium['x2'], c=color_diff, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax3.set_title('Difference')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')

cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment4-2', bbox_inches='tight')

###### Second: the 0.3/0.7 split

In [ ]:
# split data
train_scores, test_scores = train_test_split(df, test_size=0.3)

In [ ]:
contamination = 0.02
size_dataset = test_scores.shape[0]
n_outliers = math.floor(contamination*size_dataset)
t = test_scores['outlier score1'].nlargest(n=n_outliers).iloc[-1]
predictions = np.array([1 if s >= t else 0 for s in test_scores['outlier score1']])

In [ ]:
# Compute ExCeeD and Cantelli probabilities and confidences
exceed_prob1_small, exceed_conf1_small = ExCeeD_alt(train_scores['outlier score1'], test_scores['outlier score1'], predictions, contamination, alg='bayes')
exceed_prob2_small, exceed_conf2_small = ExCeeD_alt(train_scores['outlier score2'], test_scores['outlier score2'], predictions, contamination, alg='bayes')

In [ ]:
colors1 = exceed_prob1_small
colors2 = exceed_prob2_small
colors = [colors1, colors2]
color_diff = np.array(colors1) - np.array(colors2)

In [ ]:
# Visualize outlier probabilities (and the difference)
fig, (ax1,ax2,ax3) = plt.subplots(figsize=(12, 4), nrows=1, ncols=3)
p_min = min(np.concatenate(colors))
p_max = max(np.concatenate(colors))
ax1.scatter(test_scores['x1'], test_scores['x2'], c=colors1, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax1.set_title('Linear')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax2.scatter(test_scores['x1'], test_scores['x2'], c=colors2, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax2.set_title('Cubed')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
im = ax3.scatter(test_scores['x1'], test_scores['x2'], c=color_diff, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax3.set_title('Difference')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')

cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment4-3', bbox_inches='tight')

No it did not!
It would seem that the ExCeeD outlier probability estimation only evaluates based on outlier rank, not outlier score and it is not affected by splitting the data - in all three tests, there is no difference in outlier probability between the two fake outlier scores

Is that also the case for the Cantelli?

In [ ]:
# Compute Cantelli outlier probabilities
colors1 = compute_cantelli_probability_bounds(df, 'outlier score1')
colors2 = compute_cantelli_probability_bounds(df, 'outlier score2')
colors = [colors1, colors2]
color_diff = np.array(colors1) - np.array(colors2)

In [ ]:
# Visualize outlier probabilities and difference
p_min = min(np.concatenate(colors))
p_max = max(np.concatenate(colors))
fig, (ax1,ax2,ax3) = plt.subplots(figsize=(12, 4), nrows=1, ncols=3)
ax1.scatter(df['x1'], df['x2'], c=colors1, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax1.set_title('Linear')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax2.scatter(df['x1'], df['x2'], c=colors2, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax2.set_title('Cubed')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
im = ax3.scatter(df['x1'], df['x2'], c=color_diff, edgecolor='black', cmap='hot_r', vmin=p_min, vmax=p_max)
ax3.set_title('Difference')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')

cbar_ax = fig.add_axes([0.98, 0.2, 0.02, 0.65])
fig.colorbar(im, cax=cbar_ax)

fig.tight_layout(pad=3)
#fig.savefig('images/experiment4-4', bbox_inches='tight')

The cantelli approach of interpreting outlier probabilities based on oulier scores does not produce the same result for both 'outlier score1' and 'outlier score2' proving that it is in fact based on outlier score rather than outlier rank

So it seems that the ExCeeD method does not evaluate outlier scores, but rankings while the Cantelli method does evaluate scores. I need a discussion on the significance of this difference. Also maybe a discussion of why it works so well om the data sets.

Do I need another/more experiments to illustrate that the ExCeeD evaluates based on ranking and not score? E.i. show that it is not coincidence for these specific "scores" I have chosen (linear and to cubed)?

### How is the confidence expected to look?

In [ ]:
# Setup experiment parameters
n = 200
n_anom = 50
X = np.linspace(0,1,100)
p1 = X
p2 = X**2
dist_test1 = 1- binom.cdf(n - n_anom, n, p1)
dist_test2 = 1- binom.cdf(n - n_anom, n, p2)

In [ ]:
# Visualize confidence
fig, (ax1, ax2) = plt.subplots(figsize=(10,3), nrows=1, ncols=2)
ax1.plot(X, dist_test1, label='Confidence, Y=1')
ax1.plot(X, 1-dist_test1, label='Confidence, Y=0')
ax1.plot(X, [dist_test1[i] if x > 0.7 else (1-dist_test1[i]) for i,x in enumerate(X)], alpha=0.5, linewidth=5, label='Expected Confidence')
ax1.set_title('p = Relative Outlier Score')
ax1.legend(loc='center left')
ax1.set_xlabel('Relative Outlier Score')
ax1.set_ylabel('Confidence')

ax2.plot(X, dist_test2, label='Confidence, Y=1')
ax2.plot(X, 1-dist_test2, label='Confidence, Y=0')
ax2.plot(X, [dist_test2[i] if x > 0.7 else (1-dist_test2[i]) for i,x in enumerate(X)], alpha=0.55, linewidth=5, label='Expected Confidence')
ax2.set_title('p = Relative Outlier Score^2')
ax2.legend(loc='center left')
ax2.set_xlabel('Relative Outlier Score')
ax2.set_ylabel('Confidence')

#fig.savefig('images/experiment4-5', bbox_inches='tight')